In [1]:
import config
import telebot
import sys
import os
import time
import SQLighter
#from SQLighter import SQLighter
import random
import pandas as pd

import utils
import shelve
import numpy as np

from config import shelve_name, shelve_name_field

In [2]:
bot = telebot.TeleBot(config.token)

In [3]:
db_worker = SQLighter.SQLighter(config.database_name)

In [4]:
@bot.message_handler(content_types=['audio'])
def save_audio(message):
    r={
        'autor':'',
        'performer':str(message.audio.performer),
        'song':'',
        'style':'',
        'album':'',
        'year':'',
        'text':'',
        'music':str(message.audio.title),
        'id_telegram':str(message.audio.file_id),
        'playlist':'',
        'tags':'',
        'user':message.from_user.id
    }
    # в хранилище проставляем признак того, что пользователь загружает музыку
    utils.set_user_load(message.chat.id,message.from_user.id,r)
    # формиуем разметку
    markup = utils.generate_markup()
    # Отправляем предложение загрузить инфу о музыке
    bot.send_message(message.chat.id, 'расскажи мне поподробнее об этой записи. Можешь указать что-то из предложенного?',
                     reply_markup=markup, 
                     reply_to_message_id=message.message_id)

In [5]:
dict_field = {'плейлист':'playlist','исполнитель':'performer','название':'music',
                      'теги':'tags'}

In [6]:
@bot.message_handler(func=lambda message: utils.game_chek(message) and not utils.check_field(message), content_types=['text'])
def check_answer(message):
    answer = message.text
    #print('быть не должно!')
    if answer in ['плейлист','исполнитель','название','теги']:
        bot.send_message(message.chat.id, 'и это ...',
                     reply_markup=None, 
                     reply_to_message_id=message.message_id)
        with shelve.open(shelve_name) as storage:
            temp_r = storage[str(message.chat.id)+str(message.from_user.id)][1]
            temp_r[dict_field[answer]] = True
            storage[str(message.chat.id)+str(message.from_user.id)] = [storage[str(message.chat.id)+str(message.from_user.id)][0],temp_r]
        with shelve.open(shelve_name_field) as storage:
            storage[str(message.chat.id)+str(message.from_user.id)] = answer
    else:
        with shelve.open(shelve_name) as storage:
            utils.insert(config.database_name, storage[str(message.chat.id)+str(message.from_user.id)][1])
        utils.finish_user_game(message.chat.id,message.from_user.id)

In [7]:
@bot.message_handler(func=lambda message: utils.game_chek(message) and utils.check_field(message), content_types=['text'])
def add_data(message):
    with shelve.open(shelve_name_field) as storage:
        field = storage[str(message.chat.id)+str(message.from_user.id)]
    with shelve.open(shelve_name) as storage:
        temp_r = storage[str(message.chat.id)+str(message.from_user.id)][1]
        temp_r[dict_field[field]] = message.text
        storage[str(message.chat.id)+str(message.from_user.id)] = [storage[str(message.chat.id)+str(message.from_user.id)][0],temp_r]

In [8]:
with shelve.open(shelve_name) as storage:
    print (len(storage))

0


In [9]:
with shelve.open(shelve_name_field) as storage:
    print (len(storage))

0


In [10]:
db_worker.count_rows()

16

In [11]:
@bot.message_handler(commands=['next'])
def next_track(message):
    d = SQLighter.SQLighter(config.database_name)
    bot.send_audio(message.chat.id, d.random_from_playlist('new_year')[0][8])
    d.close()

In [12]:
@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, 'Просто шли /next, что бы послушать следующую новогоднюю песню!')

In [ ]:
if __name__ == '__main__':
    bot.polling(none_stop=True)